In [6]:
from sklearn.naive_bayes import GaussianNB
from sklearn import datasets
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd
from nltk.metrics import ConfusionMatrix
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.externals import joblib
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

In [7]:
emo=5

if emo == 2: 
    dataset_path = '../Data/Datasets/Binary Classification/'
else:
    dataset_path = '../Data/Datasets/Multiclass Classification/'

In [8]:
#save the classifier
train = pd.read_csv(dataset_path+'train.csv', sep=",", header=None,index_col = False)
test = pd.read_csv(dataset_path+'test.csv', sep=",", header=None,index_col = False)

whole_ds = train.append(test,ignore_index=True )
whole_ds['tokenized_sents'] = whole_ds.apply(lambda row: nltk.word_tokenize(row[0]), axis=1)

onehot_enc = MultiLabelBinarizer()
onehot_enc.fit(whole_ds['tokenized_sents'])

lsvm = LinearSVC()
lsvm = CalibratedClassifierCV(lsvm) 
lsvm.fit(onehot_enc.transform(whole_ds['tokenized_sents']), whole_ds[1])
joblib.dump(lsvm, dataset_path+'svm_classifier.pkl') 
joblib.dump(onehot_enc,dataset_path+'svm_encode.pkl')

['../Data/Datasets/Multiclass Classification/svm_encode.pkl']

In [10]:
accuracy_svm = []
folds = 10

for idx in range(1,folds+1):
    
    train = pd.read_csv(dataset_path+'cv'+str(idx)+'_train.csv', sep=",", header=None,index_col = False)
    test = pd.read_csv(dataset_path+'cv'+str(idx)+'_test.csv', sep=",", header=None,index_col = False)
    
    #train['tokenized_sents'] = train.apply(lambda _: '', axis=1)
    #for index, row in train.iterrows():
    #    train['tokenized_sents'][index] = nltk.word_tokenize(row[0])
    
    #train['tokenized_sents'] = train.apply(lambda row: nltk.word_tokenize(row[0]), axis=1)
    #test['tokenized_sents'] = test.apply(lambda row: nltk.word_tokenize(row[0]), axis=1)

    train['tokenized_sents'] = [nltk.word_tokenize(row[0]) for index, row in train.iterrows()]
    test['tokenized_sents'] = [nltk.word_tokenize(row[0]) for index, row in test.iterrows()]
    
    lsvm = LinearSVC()
    lsvm.fit(onehot_enc.transform(train['tokenized_sents']), train[1])

    score = lsvm.score(onehot_enc.transform(test['tokenized_sents']), test[1])
    accuracy_svm.append(score)
    print(score)

0.8968253968253969
0.883289124668435
0.8404255319148937
0.8829787234042553
0.8422459893048129
0.8605898123324397
0.8847184986595175
0.8337801608579088
0.8123324396782842
0.8766756032171582


In [11]:
print(accuracy_svm)

[0.8968253968253969, 0.883289124668435, 0.8404255319148937, 0.8829787234042553, 0.8422459893048129, 0.8605898123324397, 0.8847184986595175, 0.8337801608579088, 0.8123324396782842, 0.8766756032171582]


In [14]:
print(np.mean(accuracy_svm))

0.8613861280863102


In [13]:
train = pd.read_csv(dataset_path+'train.csv', sep=",", header=None,index_col = False)
test = pd.read_csv(dataset_path+'test.csv', sep=",", header=None,index_col = False)

print(test.shape)

train['tokenized_sents'] = [nltk.word_tokenize(row[0]) for index, row in train.iterrows()]
test['tokenized_sents'] = [nltk.word_tokenize(row[0]) for index, row in test.iterrows()]
    

lsvm = LinearSVC()
lsvm.fit(onehot_enc.transform(train['tokenized_sents']), train[1])

score = lsvm.score(onehot_enc.transform(test['tokenized_sents']), test[1])
print(score)

(937, 2)
0.8772678762006404
